In [5]:
def test_integral_query():
    from astropy.coordinates import SkyCoord
    import numpy as np
    from datetime import datetime
    import pandas as pd
    from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
    
    p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius="10 degree")

    cat = IntegralQuery(searchquerry)
    f = Filter(SCW_TYPE="POINTING")
    scw_ids = cat.apply_filter(f,return_coordinates=True, remove_duplicates=False)
        
    assert scw_ids.shape == (8087, 4), "The number of pointings found is not what we expect"
    
    assert np.array_equal(scw_ids[3], np.array(['003900020030', 83.635201, 22.012722,
                                                datetime(2003, 2, 7, 7, 56, 16)], dtype=object)), \
        "A content of scw_ids is not what we expect"
        
    cat.sort_by("DEC_X")
    
    assert np.array_equal(np.array(cat.table.iloc[1114]),
                          np.array(['115900130010', 1, 'POINTING', 82.235039, 16.914862, 
                                    pd.Timestamp('2012-04-10 11:09:23'), pd.Timestamp('2012-04-10 11:44:04'), 
                                    '88602500001', 'CALIBRATION', 'PUBLIC', 'PUBLIC', 2053.0, 2038.0, 
                                    2051.0, 2053.0, 2053.0, 2053.0, 2052.0, 81747968.0, 316.022])), \
                                        "Sort_by did not produce the correct table"
                                        
    f = Filter(RA_X=Range(78., None),
           DEC_X=Range(None, 30.),
           TIME=Range(min_val='2009-11-04T00:05:23', max_val='2015-09-04T00:05:23'),
           OBS_TYPE="CALIBRATION",
           PS="PUBLIC",
           PI_NAME="PUBLIC",
           GOOD_SPI=Range(None, 500))
    
    cat.sort_by("DSIZE")
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert (scw_ids.shape == (1511,)) and (scw_ids[1000]=='132700450021'), \
        "Filter gave wrong results"
                                        
    searchquerry = SearchQuery(object_name="Cyg X-1")
    cat = IntegralQuery(searchquerry)
    f = Filter()
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (19151,), "Object Query gave wrong number of pointings"
    
    f = Filter(SCW_ID='004000100010',
               SCW_VER=2)
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (0,), "SCW_ID or SCW_VER filter gave wrong result"
    
                                
test_integral_query()

In [4]:
def test_clustering():
    import numpy as np
    from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
    from IntegralPointingClustering import ClusteredQuery
    
    searchquerry = SearchQuery(object_name="Cyg X-1")
    cat = IntegralQuery(searchquerry)
    f = Filter(SCW_TYPE="POINTING")
    scw_ids = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)
    
    np.random.seed(0)
    
    cq = ClusteredQuery(scw_ids, 1/6, 1/15, 1, 0.3,
                        cluster_size_range = (5,5),
                        cluster_size_preference_threshold = (),
                        failed_improvements_max=2,
                        suboptimal_cluster_size = 2,
                        close_suboptimal_cluster_size = 3,
                        track_performance=True)
    
    region_sizes = [len(cq.clusters[i+1]) for i in range(5)]
    
    assert region_sizes == [440, 123, 56, 73, 1710], "Cluster Sizes are not what we expect"
    
    
    clustered = set()
    n = 0
    for key, value in cq.clusters.items():
        for c in value:
            clustered = clustered | set(c.indices)
            n += c.num_pointings
            
    missing = [i for i in range(cq._num_pointings) if i not in clustered]
    
    assert missing == [], "Not all pointings were clustered"
    
    assert cq._num_pointings == n, "Some pointings are in multiple clusters"
    
    ss = cq.get_clustered_scw_ids()
    
    assert (ss[1][0], ss[5][-1]) == (['001130001060'], 
                                     ['250300020010',
                                      '250300090010',
                                      '250300100010',
                                      '250300110010',
                                      '250300120010']), \
                                        "The clusters are not what we expect"
    
    
test_clustering()

Performance Review!

Run Time:
40.14s

Region Sizes:
[(1, 129), (2, 32), (3, 4), (4, 7), (5, 1), (6, 5), (7, 3), (9, 2), (11, 2), (13, 1), (14, 2), (15, 1), (17, 2), (21, 2), (24, 2), (26, 1), (27, 1), (28, 2), (41, 1), (54, 2), (57, 1), (58, 3), (59, 3), (101, 2), (120, 1), (126, 1), (146, 1), (150, 1), (160, 2), (161, 1), (182, 1), (204, 2), (235, 2), (238, 1), (268, 1), (273, 1), (284, 1), (292, 2), (306, 1), (369, 1), (393, 1), (431, 1), (433, 1), (497, 1), (541, 1), (783, 1), (836, 1)]

Initial Cluster Sizes:
{1: 483, 2: 124, 3: 52, 4: 61, 5: 1713}

Final Cluster Sizes:
{1: 440, 2: 123, 3: 56, 4: 73, 5: 1710}

Attempted Improvements:
140

Implemented Improvements:
56

Dead-End Clustering Paths:
7


In [3]:
import numpy as np
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
from IntegralPointingClustering import ClusteredQuery

searchquerry = SearchQuery(object_name="Cyg X-1")
cat = IntegralQuery(searchquerry)
f = Filter(SCW_TYPE="POINTING")
scw_ids = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)

np.random.seed(0)

cq = ClusteredQuery(scw_ids, 0, 1/15, 1, 0.3, 6,
                    cluster_size_range = (5,5),
                    cluster_size_preference_threshold = (),
                    failed_improvements_max=2,
                    suboptimal_cluster_size = 2,
                    close_suboptimal_cluster_size = 3,
                    track_performance=True)

Performance Review!

Run Time:
50.19s

Region Sizes:
[(1, 95), (2, 22), (3, 4), (4, 4), (5, 1), (6, 5), (7, 2), (8, 1), (9, 1), (11, 2), (14, 2), (15, 1), (17, 2), (20, 1), (21, 1), (24, 2), (26, 1), (27, 1), (28, 2), (30, 1), (43, 1), (54, 2), (57, 1), (58, 1), (59, 4), (72, 1), (101, 1), (125, 1), (126, 1), (146, 1), (150, 1), (160, 2), (162, 1), (182, 1), (205, 1), (213, 1), (235, 1), (238, 1), (243, 1), (268, 1), (273, 1), (284, 1), (292, 2), (306, 1), (369, 1), (393, 1), (431, 1), (448, 1), (499, 1), (642, 1), (783, 1), (836, 1)]

Initial Cluster Sizes:
{1: 420, 2: 130, 3: 58, 4: 48, 5: 1730}

Final Cluster Sizes:
{1: 399, 2: 121, 3: 55, 4: 60, 5: 1730}

Attempted Improvements:
123

Implemented Improvements:
42

Dead-End Clustering Paths:
6
